<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Implementing-your-first-Go-bot" data-toc-modified-id="Implementing-your-first-Go-bot-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Implementing your first Go bot</a></span><ul class="toc-item"><li><span><a href="#Using-an-enum-to-represent-players" data-toc-modified-id="Using-an-enum-to-represent-players-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Using an enum to represent players</a></span></li><li><span><a href="#Using-Tuples-to-represent-points-of-a-Go-board" data-toc-modified-id="Using-Tuples-to-represent-points-of-a-Go-board-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Using Tuples to represent points of a Go board</a></span></li></ul></li></ul></div>

# Deep Learning and the Game of Go

Max Pumperla <br>
Kevin Ferguson <br>

Notes by Michael Ruggiero

## Implementing your first Go bot

I had some trouble following along with the book, so I made this notebook to help myself piece the code together. The book tends to introduce a module, and then comes back to it later. While everything is reasonably labeled, I had a little trouble doubling back so I am going to run through all of the modules part by part.

Here is the basic outline of the code used in chapter 3, with the pycache and .pyc complied work deleted.

.<br>
├── bot_v_bot.py<br>
├── dlgo<br>
│   ├── agent<br>
│   ├── goboard_fast.py<br>
│   ├── goboard_fast.pyc<br>
│   ├── goboard_fast_test.py<br>
│   ├── goboard.py<br>
│   ├── goboard.pyc<br>
│   ├── goboard_slow.py<strong> (Section 3.3)</strong><br>
│   ├── goboard_test.py<br>
│   ├── gotypes.py <strong>(Section 3.1 - 3.2)</strong><br> 
│   ├── __init__.py<br>
│   ├── scoring.py<br>
│   ├── utils.py<br>
│   └── zobrist.py<br>
├── generate_zobrist.py<br>
├── human_v_bot.py<br>
├── requirements.txt<br>
└── setup.py<br>

### Using an enum to represent players

### Using Tuples to represent points of a Go board

In [7]:
import enum
from collections import namedtuple

__all__ = [
    'Player',
    'Point']


class Player(enum.Enum):
    black = 1
    white = 2

    @property
    def other(self):
        return Player.black if self == Player.white else Player.white

class Point(namedtuple('Point', 'row col')):
    #This sets the points of a neighbor on the board.
    def neighbors(self):
        
        return [
            Point(self.row - 1, self.col),
            Point(self.row + 1, self.col),
            Point(self.row, self.col - 1),
            Point(self.row, self.col + 1),
        ]

    def __deepcopy__(self, memodict={}):
        # These are very immutable.
        return self